In [184]:
# Improting Geo Json
from pymongo import MongoClient
import pandas as pd
import requests
import os
from dotenv import load_dotenv
load_dotenv()

client = MongoClient('mongodb://localhost:27017/')
db = client.GeoComps
a=db.GeoComps.find()
b=pd.DataFrame(a)
b["headquarters"].head()
#b["headquarters"]["coordinates"]
b.shape

(2939, 10)

In [185]:
b.drop(columns=['_id'], inplace=True)
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75


In [186]:
radio=2000
geopoint=b["headquarters"][0]
geopoint

{'type': 'Point', 'coordinates': [-118.393064, 34.090368]}

In [187]:
def findNear(geopoint,radio):
     return list(db.GeoComps.find({
        "headquarters": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio,
         }
       }
    }))
geopoint=b["headquarters"][0]
lst=findNear(geopoint,radio)
lst

[{'_id': ObjectId('5d2759fa6497a5d8bff314d3'),
  'name': 'Geni',
  'lat': 34.090368,
  'lng': -118.393064,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]},
  'number_of_employees': 18,
  'category_code': 'web',
  'founded_year': 2006,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31ba1'),
  'name': 'MyFreeImplants',
  'lat': 34.087825,
  'lng': -118.379493,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.379493, 34.087825]},
  'number_of_employees': 2,
  'category_code': 'network_hosting',
  'founded_year': 2005,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31b66'),
  'name': 'Meteor Games',
  'lat': 34.09316,
  'lng': -118.378335,
  'country': 'USA',
  'headquarters': {'type': 'Point', 'coordinates': [-118.378335, 34.09316]},
  'number_of_employees': 70,
  'category_code': 'games_video',
  'founded_year': 2007,
  'age': 'Old'},
 {'_id': ObjectId('5d2759fa6497a5d8bff31d19'),
  'name': 'Book

In [188]:
def sum_employees(lst):
    suma=0
    for i in lst:
        suma+= i["number_of_employees"]
    return suma

n=sum_employees(lst)
n


97

In [189]:
sum_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=findNear(geopoint,radio)
    n=sum_employees(lost)
    sum_column.append(n)


In [190]:
b["Employees_Near"]=sum_column

In [191]:
b.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,97
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,2183
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35,180
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012


In [192]:
def new_old(lst):
    old=0
    new=0
    for i in lst:
        if i["age"]=='Old':
            old+=1
        else:
            new+=1
     
    if old>0:
        ratio= new/old
        return ratio
    else:
        return 1.5



In [193]:
ratio_column=[]
for i in range(len(b)):
    geopoint=b["headquarters"][i]
    lost=list(findNear(geopoint,radio))
    n=new_old(lost)
    ratio_column.append(n)


In [194]:
b["New/Old_Ratio"]=ratio_column

In [195]:
b.head(10)

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
0,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,Geni,18,97,0.000000
1,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,2183,0.000000
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869,0.272727
3,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,Jingle Networks,35,180,0.000000
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012,0.393939
5,Old,games_video,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,272,0.111111
6,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.0899512...",37.423390,-122.089951,Jajah,110,1234,0.090909
7,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-74.3372, 40...",40.801358,-74.337200,AdaptiveBlue,15,15,0.000000
8,Old,web,USA,1995,"{'type': 'Point', 'coordinates': [-121.930035,...",37.295005,-121.930035,eBay,15000,15578,0.000000
9,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-122.07948, ...",37.392936,-122.079480,Ustream,250,2065,0.166667


In [196]:
b1=b[b['New/Old_Ratio'] != 0]
b1.head()

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
2,Old,web,USA,2005,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253,Wetpaint,47,869,0.272727
4,Old,web,USA,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,Meetup,75,2012,0.393939
5,Old,games_video,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,272,0.111111
6,Old,mobile,USA,2005,"{'type': 'Point', 'coordinates': [-122.0899512...",37.423390,-122.089951,Jajah,110,1234,0.090909
9,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-122.07948, ...",37.392936,-122.079480,Ustream,250,2065,0.166667


In [197]:
print(b1.shape)


(1685, 11)


In [198]:
b1['country'].value_counts().head()

USA    1124
GBR     115
CAN      88
FRA      55
DEU      50
Name: country, dtype: int64

In [199]:
b1=b1[b1['country'] == 'USA']
print(b1.shape)
b1.head()
b1.describe()

(1124, 11)


,founded_year,lat,lng,number_of_employees,Employees_Near,New/Old_Ratio
count,1124.000000,1124.000000,1124.000000,1124.000000,1124.000000,1124.000000
mean,2005.914591,38.477503,-102.759413,180.303381,3002.714413,0.684808
std,3.413842,4.481410,23.540114,3084.747416,10281.233931,0.538107
min,1990.000000,17.415429,-159.480262,1.000000,1.000000,0.047619
25%,2005.000000,37.320643,-122.265594,3.000000,41.000000,0.319444
50%,2007.000000,37.783662,-118.330006,9.000000,501.500000,0.428571
75%,2008.000000,40.747071,-77.231374,30.000000,2039.250000,1.000000
max,2013.000000,61.216583,78.434422,99999.000000,100961.000000,4.000000


In [200]:
new_old_upper=b1['New/Old_Ratio'].quantile(0.75)
print(new_old_upper)
# FIRST I keep TOP 50% of the New Old Ratio and then the TOP 50% of the Employees Near


1.0


In [201]:
b2=b1[b1['New/Old_Ratio'] >= new_old_upper]
b2.shape
b2.head(10)

,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
16,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Mahalo,40,1197,1.166667
40,Old,network_hosting,USA,2005,"{'type': 'Point', 'coordinates': [-122.143701,...",37.425801,-122.143701,Box,950,1041,1.000000
66,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,IntenseDebate,4,46,1.000000
89,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,Filtrbox,14,46,1.000000
100,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.495025,...",34.018275,-118.495025,Docstoc,40,279,1.750000
106,New,games_video,USA,2008,"{'type': 'Point', 'coordinates': [-122.151158,...",47.597965,-122.151158,YouBeQB,8,4408,1.000000
122,New,mobile,USA,2010,"{'type': 'Point', 'coordinates': [-111.9035, 3...",33.817100,-111.903500,GENWI,25,25,1.500000
136,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-121.983814,...",37.222604,-121.983814,Houseglue,5,27,1.000000
178,Old,games_video,USA,2005,"{'type': 'Point', 'coordinates': [-83.734673, ...",42.292649,-83.734673,Zattoo,45,79,3.000000
191,Old,games_video,USA,2007,"{'type': 'Point', 'coordinates': [-118.468672,...",34.064139,-118.468672,GotCast,5,28,1.000000


In [202]:
employee_median=b2['Employees_Near'].median()
print(employee_median)
b2=b2[b2['Employees_Near'] >= employee_median]
print(b2.shape)
b2.head()

13.0
(173, 11)


,age,category_code,country,founded_year,headquarters,lat,lng,name,number_of_employees,Employees_Near,New/Old_Ratio
16,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Mahalo,40,1197,1.166667
40,Old,network_hosting,USA,2005,"{'type': 'Point', 'coordinates': [-122.143701,...",37.425801,-122.143701,Box,950,1041,1.000000
66,Old,web,USA,2006,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,IntenseDebate,4,46,1.000000
89,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-105.276843,...",40.010492,-105.276843,Filtrbox,14,46,1.000000
100,Old,web,USA,2007,"{'type': 'Point', 'coordinates': [-118.495025,...",34.018275,-118.495025,Docstoc,40,279,1.750000


In [203]:
# With 287 companies left, I export the results to tableau to filter deeper in a map and adding API data. 
b2.to_csv('geo_clean.csv')


In [204]:
#in san francisco 37.7749° N, 122.4194° W
key=os.environ['API_Token']
geocode_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.4419,-122.1430&radius=49900&keyword=Starbucks&key={}".format(key)
res = requests.get(geocode_url)
results = res.json()
results






{'html_attributions': [],
 'next_page_token': 'CqQCIAEAAL_Il3VopzCAj6G_W6pX9En0qhia_rb1hwnytTq3h700MJ6OcV6QBfGliXCVYQr0pU286qJbotrX6v9i_NV12mFIb4-YkUlQZFW7q2w_rc_QiGifRQz9Yk4KcmcjHisjEeZp2z_GgoOjT3N7Fx4D6C5f-beVkqO82ngFCxLm8Yv227Rq5QaN9hiNZDSfA28teK6gcMh3miXANlkzoWb_aHKKk3jiq24_lmu7d4cMWUm7YcUiF4tO3-naSIVQSdx9HQD5QQuciB9wEJmB9uFpEE2g7VUjJr-VOzycIn9NleJYIWdYyEYtG7DFjuvbPB7VZe4rmXmIcyMqWDJg0Esk9942KPSa-6DA5Hwu2KtiBdK-OgY0PdRIC_CpYnoNLj9HvRIQqwUWkK1Ipti9oWkH_nGuVxoUvy8meuCkAtKZiBE_9lHaYR5FjAk',
 'results': [{'geometry': {'location': {'lat': 37.427381, 'lng': -122.149213},
    'viewport': {'northeast': {'lat': 37.42872647989272,
      'lng': -122.1478205701073},
     'southwest': {'lat': 37.42602682010727, 'lng': -122.1505202298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
   'id': 'dc76f5716b8fd67e7d76ec1115728a7a8e04f82f',
   'name': 'Starbucks',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 4032,
     'html_attributions': ['<a hre

In [205]:
results.to_json('starbucks.json', orient='records')

AttributeError: 'dict' object has no attribute 'to_json'